In [1]:
import numpy as np
x = np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [2]:
print(x[0].shape)
print(x[1].shape)
print(x[0,0])

(1, 28, 28)
(1, 28, 28)
[[4.12047975e-01 3.42317924e-02 5.96055937e-03 3.39096989e-01
  3.00208795e-01 7.25163456e-01 2.98755821e-01 1.27656312e-01
  9.07581886e-01 9.92008791e-01 9.16094435e-01 3.61200431e-01
  1.35252818e-01 8.81584009e-01 3.95998140e-01 3.62939198e-01
  5.05374148e-01 2.67895674e-01 9.88024321e-01 9.70314257e-01
  8.73675745e-01 2.13197839e-01 5.73343767e-01 4.45980087e-01
  4.57194767e-01 1.46284282e-01 2.44382780e-01 4.00894064e-01]
 [3.73972593e-01 9.01185098e-01 3.90441356e-01 2.78494101e-01
  3.92671644e-02 1.14366649e-02 6.07810506e-01 2.49024750e-01
  5.10303694e-01 5.51788958e-02 3.60328791e-01 8.09013942e-01
  9.25074862e-01 2.59219893e-01 8.39996072e-01 2.19459416e-01
  1.33458544e-01 1.31517557e-01 1.40974298e-01 1.45018170e-01
  8.56484272e-01 7.66453251e-01 8.71824811e-01 6.05604231e-02
  7.82606626e-01 8.22902688e-01 5.85891254e-01 1.52652355e-01]
 [5.32928174e-01 3.51943121e-02 2.52161114e-01 8.37938774e-01
  3.82343495e-01 7.07281484e-01 8.70467656e-

# im2col method

입력 받은 이미지를 2차원 배열로 변환

In [3]:
def im2col(input_data, filter_h, filter_w, stride=1,pad=0):
    """
    파라미터
    input_data : 4차원 배열(이미지수, 채널수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    """
    
    N,C,H,W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride+1
    out_w = (W + 2*pad - filter_w)//stride+1

    img = np.pad(input_data, [(0,0),(0,0),(pad,pad),(pad,pad)],'constant')
    col = np.zeros((N,C,filter_h, filter_w, out_h, out_w))
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:,:,y,x,:,:] = img[:, :, y:y_max:stride, x:x_max:stride]
            
        col = col.transpose(0,4,5,1,2,3).reshape(N*out_h * out_w, -1)
        return col

# col2im method

In [7]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """
    파라미터
    col : 2차원배열
    input_shape : 원래 이미지 데이터의 형상
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    """
    
    N,C,H,W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    
    col = col.reshape(N,out_h, out_w, C,filter_h,filter_w).transpose(0,3,4,5,1,2)
    img = np.zeros((N,C, H + 2*pad + stride-1,W + 2*pad + stride-1))
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:,:,y,x,:,:] = img[:, :, y:y_max:stride, x:x_max:stride]
            
    return img[:,:,pad:pad+H,pad:pad+W]

# Conv2D layer

In [4]:
x = np.random.randn(1,1,3,3)
print(x,'\n')

pad = 1
img = np.pad(x ,[ (0,0) , (0,0) , (pad,pad) , (pad,pad) ] , 'constant')
print(img[0,0])

[[[[-0.66957921 -1.1802164   0.63783729]
   [ 1.55501834  0.40892521 -0.25895331]
   [ 0.11957548  0.95561677 -0.16577696]]]] 

[[ 0.          0.          0.          0.          0.        ]
 [ 0.         -0.66957921 -1.1802164   0.63783729  0.        ]
 [ 0.          1.55501834  0.40892521 -0.25895331  0.        ]
 [ 0.          0.11957548  0.95561677 -0.16577696  0.        ]
 [ 0.          0.          0.          0.          0.        ]]


In [6]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        FN, C, FH, FW = self.W.shape
        """
        FN : 필터의 개수
        C : 필터의 깊이
        FH : 필터의 높이
        FW : 필터의 너비
        """
        
        N, C, H, W = x.shape
        """
        N : 입력 데이터의 개수
        C : 입력 데이터의 깊이
        H : 입력 데이터의 높이
        W : 입력 데이터의 너비
        """
        
        
        out_h = (H + 2*self.pad - FH)//self.stride + 1
        out_w = (H + 2*self.pad - FW)//self.stride + 1
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        
        out = np.dot(col,col_W) + self.b
        
        out = out.reshape(N, out_h,out_w,-1).transpose(0,3,1,2)
        
        return out

# Pooling layer